In [119]:
import pandas as pd

In [120]:
from preprocessing.logParser import openLogs
from content.video import Video

In [121]:
cursor = openLogs('logs/spbu_ACADRU_spring_2018-TL')

In [122]:
request = """
    select *
    from play_video
"""

cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)
df.head(5)

,user_id,course_name,execute_time,page,video_time
0,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:11:17.384343+00:00,https://courses.openedu.ru/courses/course-v1:s...,0.008867
1,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:11:40.543950+00:00,https://courses.openedu.ru/courses/course-v1:s...,22.540000
2,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:37.350351+00:00,https://courses.openedu.ru/courses/course-v1:s...,0.000000
3,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:46.876583+00:00,https://courses.openedu.ru/courses/course-v1:s...,8.960000
4,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:49.363691+00:00,https://courses.openedu.ru/courses/course-v1:s...,36.000000


In [123]:
def get_videos_pages():
    """get the set of pages y load_video event"""
    
    request = """
            select *
            from load_video
            """
    cursor.execute(request)
    data = cursor.fetchall()
    
    columns_names = []
    for i in cursor.description[:]:
        columns_names.append(i[0])
    
    df = pd.DataFrame(data=data, columns=columns_names)
    
    pages = set()
    for page in df.page:
        pages.add(page)
    return pages

In [124]:
def check(page):
    video_events = ["hide_transcript", "pause_video",
                    "play_video", "seek_video",
                    "show_transcript", "speed_change_video", "stop_video",
                    "video_hide_cc_menu", "video_show_cc_menu"]

    for i in video_events:
        request = """
                    select *
                    from {}
                    where page = "{}"
                    """.format(i, page)
        cursor.execute(request)
        data = cursor.fetchall()
        if data : return True
    return False

In [125]:
videos = []
for page in get_videos_pages():
    video = Video(page)
    videos.append(video)
    
    # get length by stop_video event
    request = """
            select max(video_time)
            from stop_video
            where page = "{}"
            """.format(page)
    cursor.execute(request)
    length = cursor.fetchall()[0]
    length = length[0]
    if length == None: 
        if check(page):
            print(page)

    request = """
            select *
            from seek_video
            where page = "{}"
            """.format(page)
    cursor.execute(request)
    data = cursor.fetchall()
    columns_names = []
    for i in cursor.description[:]:
        columns_names.append(i[0])
    df = pd.DataFrame(data=data, columns=columns_names)
    df[df.video_new_time < df.video_old_time]

https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/64a281fe82e642dd9c9cef40b8e788ec/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/3a57e4c7f4b1485f940ed66ddcd02039/f1e6ed753f2f49abaf573baee5a39326/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/30de86d10c05431988f0934a2cb36337/f20cf3906ad14c269c53a65c9fb49ca6/?child=last
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/1282161218d447858cfac1ca9bad94f0/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/d331d71c92f34c28bb1793382368f8b8/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/04a2206dce574325be4b853171e3aca1/59e2b73844b046d2ad9f8f6fab021bad/?child=first
https://courses.openedu.ru/courses/course-v1:sp